# Hands on Example - Simulating Bond

In [303]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
import simulate_multi_cty_bond as smcb
import simulate_bond as sb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define Variables

In [ ]:
#define nominals
nominal_sng = 100000000
nominal_mlt = round(nominal_sng + nominal_sng)

#define premium and risk free rate
premium = 0.05
rf_rate = 0.00

#number countries
countries = [1,2]

#specify if average annual returns or compounded annual returns are wanted
ann_com = True

#specify payouts and damages per event
pay_dam_df_c1 = pd.DataFrame([{'pay': np.array(140.0), 'damage': np.array(150.0), 'year': 0, 'month': np.array(2)},
                              {'pay': np.array(50.0), 'damage': np.array(50.0), 'year': 5, 'month': np.array(3)}])

pay_dam_df_c2 = pd.DataFrame([{'pay': np.array(40.0), 'damage': np.array(35.0), 'year': 1, 'month': np.array(10)},
                              {'pay': np.array(160.0), 'damage': np.array(200.0), 'year': 5, 'month': np.array(9)}])

pay_dam_df_dic = {1: pay_dam_df_c1,
                  2: pay_dam_df_c2}

nominal_dic = {1: nominal_sng,
               2: nominal_sng}

## Simulate Single-Country Bonds

### Country 01

In [305]:
exp_loss_pool_c1, att_prob_pool_c1, df_loss_month_c1, es_metrics_c1 = sb.init_exp_loss_att_prob_simulation(pay_dam_df_c1, nominal_sng)
metrics_pool_c1, returns_pool_c1 = sb.init_bond_simulation(pay_dam_df_c1, premium, rf_rate, nominal_sng, ann_com)

Expected Loss = 3.8738738738738736e-08
Attachment Probability = 0.04504504504504504
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
Nominal:  99999860.0
Payout:  [array(140.)]
NCF empty:  4999993.0
Nominal:  99999860.0
Payout:  0
Orig Nominal:  100000000
Nominal:  99999860.0
Payout:  [array(140.)]
NCF empty:  4999993.0
Nominal:  99999860.0
Payout:  0
NCF empty:  4999993.0
Nominal:  99999860.0
Payout:  0
Orig Nominal:  100000000
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
Orig Nominal:  100000000
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
Nominal:  99999950.0
Payout:  [array(50.)]
Orig Nominal:  100000000
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
Nominal:  99999950.0
Payout:  [array(50.)]
NCF empty:  4999997.5
Nominal:  99999950.0
Payout:  0
Orig Nominal:  100000000
Nominal:  99999950.0
Payout:  

In [306]:
es_metrics_c1

{'VaR_99_ann': 1.310000000000005e-06,
 'VaR_99_tot': 1.4e-06,
 'VaR_95_ann': 0.0,
 'VaR_95_tot': 6.799999999999961e-07,
 'ES_99_ann': 1.4e-06,
 'ES_99_tot': nan,
 'ES_95_ann': 8.599999999999999e-07,
 'ES_95_tot': 1.4e-06}

In [307]:
metrics_pool_c1

,Premium,Sharpe_ratio_ann,Sharpe_ratio_tot,Coverage,Basis_risk,Average Payments,Summed Payments,Total Premiums
0,0.05,246106.928367,413420.048331,0.955556,-20.0,86.0,430.0,5.550000e+08


### Country 02

In [308]:
exp_loss_pool_c2, att_prob_pool_c2, df_loss_month_c1, es_metrics_c2 = sb.init_exp_loss_att_prob_simulation(pay_dam_df_c2, nominal_sng)
metrics_pool_c2, returns_pool_c2 = sb.init_bond_simulation(pay_dam_df_c2, premium, rf_rate, nominal_sng, ann_com)

Expected Loss = 9.369369369369368e-08
Attachment Probability = 0.07207207207207207
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
Nominal:  99999960.0
Payout:  [array(40.)]
NCF empty:  4999998.0
Nominal:  99999960.0
Payout:  0
Orig Nominal:  100000000
Nominal:  99999960.0
Payout:  [array(40.)]
NCF empty:  4999998.0
Nominal:  99999960.0
Payout:  0
NCF empty:  4999998.0
Nominal:  99999960.0
Payout:  0
Orig Nominal:  100000000
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
Nominal:  99999840.0
Payout:  [array(160.)]
Orig Nominal:  100000000
NCF empty:  5000000.0
Nominal:  100000000
Payout:  0
Nominal:  99999840.0
Payout:  [array(160.)]
Nominal:  99999680.0
Payout:  [array(160.)]
Orig Nominal:  100000000
Nominal:  99999840.0
Payout:  [array(160.)]
Nominal:  99999680.0
Payout:  [array(160.)]
NCF empty:  4999984.0
Nominal:  99999680.0
Payout:  0
Orig Nominal:  100000000
Nominal:  99999840.0
Payout:  [array(160.)]
NCF empty:  49

In [309]:
es_metrics_c2

{'VaR_99_ann': 1.6e-06,
 'VaR_99_tot': 3.2e-06,
 'VaR_95_ann': 1e-06,
 'VaR_95_tot': 1.919999999999993e-06,
 'ES_99_ann': nan,
 'ES_99_tot': nan,
 'ES_95_ann': 1.6e-06,
 'ES_95_tot': 3.2e-06}

In [310]:
metrics_pool_c2

,Premium,Sharpe_ratio_ann,Sharpe_ratio_tot,Coverage,Basis_risk,Average Payments,Summed Payments,Total Premiums
0,0.05,133562.665071,178942.881255,0.818898,-230.0,173.333333,1040.0,5.549999e+08


## Simulate Multi-Country Bond

In [311]:
exp_loss_pool, att_prob_pool, df_loss_month, total_losses, es_metrics, MES_cty = smcb.init_exp_loss_att_prob_simulation(countries, pay_dam_df_dic, nominal_mlt, nominal_dic)
metrics_pool, returns_pool, tot_coverage_pool_cty = smcb.init_bond_simulation(pay_dam_df_dic, premium, rf_rate, nominal_mlt, countries, nominal_dic)

Expected Loss = 6.62162162162162e-08
Attachment Probability = 0.07207207207207207
NCF empty:  10000000.0
Nominal:  200000000
Payout:  0
Nominal:  199999820.0
Payout:  [array(40.), array(140.)]
NCF empty:  9999991.0
Nominal:  199999820.0
Payout:  0
NCF list:  [10000000.0, 9999817.75, 9999991.0]
Nominal:  199999820.0
Payout:  [array(40.), array(140.)]
NCF empty:  9999991.0
Nominal:  199999820.0
Payout:  0
NCF empty:  9999991.0
Nominal:  199999820.0
Payout:  0
NCF list:  [9999817.75, 9999991.0, 9999991.0]
NCF empty:  10000000.0
Nominal:  200000000
Payout:  0
NCF empty:  10000000.0
Nominal:  200000000
Payout:  0
Nominal:  199999840.0
Payout:  [array(160.)]
NCF list:  [10000000.0, 10000000.0, 9999838.0]
NCF empty:  10000000.0
Nominal:  200000000
Payout:  0
Nominal:  199999840.0
Payout:  [array(160.)]
Nominal:  199999630.0
Payout:  [array(50.), array(160.)]
NCF list:  [10000000.0, 9999838.0, 9999778.125]
Nominal:  199999840.0
Payout:  [array(160.)]
Nominal:  199999630.0
Payout:  [array(50.),

In [312]:
es_metrics

{'VaR_99_ann': 1.05e-06,
 'VaR_99_tot': 1.85e-06,
 'VaR_95_ann': 8e-07,
 'VaR_95_tot': 1.2099999999999965e-06,
 'ES_99_ann': nan,
 'ES_99_tot': nan,
 'ES_95_ann': 9.9e-07,
 'ES_95_tot': 1.85e-06}

In [313]:
MES_cty

{1: {'95': 4.2999999999999996e-07, '99': nan},
 2: {'95': 5.599999999999999e-07, '99': nan}}

In [314]:
metrics_pool

,Premium,Sharpe_ratio_ann,Sharpe_ratio_tot,Coverage,Basis_risk,Average Payments,Summed Payments,Total Premiums
0,0.05,203578.162486,288407.606285,0.854651,-250.0,245.0,1470.0,1.110000e+09


In [315]:
print(metrics_pool['Total Premiums'])
print(metrics_pool_c1['Total Premiums'] + metrics_pool_c2['Total Premiums'])

0    1.110000e+09
Name: Total Premiums, dtype: float64
0    1.110000e+09
Name: Total Premiums, dtype: float64
